In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_config, get_peft_model, LoraConfig

model = AutoModelForCausalLM.from_pretrained("D:\CoachBot\SFTuned")
tokenizer=AutoTokenizer.from_pretrained("D:\CoachBot\SFTuned", use_fast=False)

In [5]:
from peft import PeftModel
import torch
model = PeftModel.from_pretrained(
    model,
    "./DPO_1",                    
    torch_dtype=torch.float16,    
).to("cuda")                      

tokenizer = AutoTokenizer.from_pretrained(
    "./DPO_1",                    
    trust_remote_code=True,       
)


In [6]:
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig
ds = load_dataset("entfane/psychotherapy-dpo")
#ds = load_dataset("hungtuchen/MentalLlama-dpo-pairs")

In [ ]:
'''ds = ds.rename_column("Context", "prompt")
print(ds["train"].column_names)  # 应该显示 ['prompt','chosen','rejected']'''

['prompt', 'chosen', 'rejected']


In [7]:
ds = ds.remove_columns("illness")
ds = ds.rename_column("input", "prompt")
print(ds["train"].column_names)  # 应该显示 ['prompt','chosen','rejected']

['prompt', 'chosen', 'rejected']


In [8]:
len(ds["train"])

168

In [9]:
lora_config = LoraConfig(
    r=8,                      
    target_modules=["q_proj","v_proj", "k_proj", "o_proj", "dense_h_to_4h"],  
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, lora_config)

n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("可训练参数总数：", n_trainable)

可训练参数总数： 2293760


In [10]:
training_args = DPOConfig(
    output_dir="./checkpoints",            
    per_device_train_batch_size=2,
    gradient_accumulation_steps=6,
    learning_rate=1e-5,
    num_train_epochs=3,
    logging_steps=5,
    save_strategy="steps",
    save_steps=5,
    save_total_limit=2, 
    gradient_checkpointing=True

)
model.gradient_checkpointing_enable()
trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],   
    tokenizer=tokenizer,
)

C:\Users\AriZu\AppData\Local\Temp\ipykernel_14592\2156461788.py:15: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  trainer = DPOTrainer(


Applying chat template to train dataset:   0%|          | 0/168 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/168 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
trainer.train()

model.save_pretrained("./DPO_2")
tokenizer.save_pretrained("./DPO_2")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
5,0.584600
10,0.369000
15,0.236200
20,0.148600
25,0.108900
30,0.093400
35,0.061400
40,0.056800


c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
c:\Users\AriZu\.conda\envs\train\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inpu

('./DPO_2\\tokenizer_config.json',
 './DPO_2\\special_tokens_map.json',
 './DPO_2\\vocab.json',
 './DPO_2\\merges.txt',
 './DPO_2\\added_tokens.json',
 './DPO_2\\tokenizer.json')